In [128]:
import pandas as pd
import math
from sqlalchemy import create_engine

In [129]:
df = pd.read_html("http://zipatlas.com/us/zip-code-comparison/population-density.1.htm")

In [99]:
#saving column names
column = df[10][:1].values.tolist()[0]
column

['#',
 'Zip Code',
 'Location',
 'City',
 'Population',
 'People / Sq. Mile',
 'National Rank']

In [100]:
zip_df = pd.DataFrame()
for p in range(1,318+1):
    #10th table of site and reading from 2nd row since first row returns columns
    apnd = pd.read_html(f"http://zipatlas.com/us/zip-code-comparison/population-density.{p}.htm")[10][1:]
    #set columns
    apnd.columns = column
    #concat to empty dataframe created before for loop
    zip_df = pd.concat([zip_df,apnd])
    print(p)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


In [101]:
#selecting columns to conserve
zip_df = zip_df[["Zip Code","Location","Population","People / Sq. Mile"]]

In [102]:
#renaming columns
zip_df.columns = ["zip","coordinates","population","density"]

In [103]:
#setting population and density columns as numeric values for calculation
zip_df["population"] = zip_df["population"].astype(int)
zip_df["density"] = zip_df["density"].astype(float)
zip_df.dtypes

zip             object
coordinates     object
population       int32
density        float64
dtype: object

In [104]:
#population density = population / area
zip_df["area"] = zip_df["population"] / zip_df["density"]

In [105]:
#area = pi r^2
zip_df["radius"] = (zip_df["area"]/3.14)**(.5)

In [106]:
#sort values
zip_df.sort_values("zip",inplace= True)

In [107]:
#creating sqlite database to store
con = create_engine("sqlite:///zip_data0.sqlite")
zip_df.to_sql("area",con,if_exists="replace",index=False)
#also as in csv for backup
zip_df.to_csv("zip_area.csv",index=False)

In [137]:
area = pd.read_sql("area",con)
city = pd.read_sql("city",con)

In [138]:
df = pd.merge(area,city,on="zip")
df

,zip,coordinates,population,density,area,radius,city,state,lat,lon
0,00601,"18.181560, -66.758147",19143,285.19,67.123672,4.623523,Adjuntas,PR,18.16,-66.72
1,00602,"18.358577, -67.174708",42042,1359.03,30.935299,3.138791,Aguada,PR,18.38,-67.18
2,00603,"18.452658, -67.125609",55530,1830.95,30.328518,3.107855,Aguadilla,PR,18.43,-67.15
3,00604,"18.492373, -67.138234",3923,1279.94,3.064987,0.987983,Aguadilla,PR,18.43,-67.15
4,00606,"18.173904, -66.935931",6449,176.02,36.637882,3.415862,Maricao,PR,18.18,-66.98
5,00610,"18.293389, -67.120670",27975,748.64,37.367760,3.449719,Anasco,PR,18.28,-67.14
6,00612,"18.412693, -66.703445",72730,965.30,75.344453,4.898474,Arecibo,PR,18.45,-66.73
7,00616,"18.403670, -66.664759",10532,666.66,15.798158,2.243047,Bajadero,PR,18.42,-66.67
8,00617,"18.437505, -66.559407",23370,1060.63,22.034074,2.649004,Barceloneta,PR,18.45,-66.56
9,00622,"17.988103, -67.160357",8104,258.55,31.344034,3.159458,Boqueron,PR,17.99,-67.15


In [191]:
rent = pd.read_csv("Zip_ZriPerSqft_AllHomes.csv")

In [192]:
rent.rename(columns={"RegionName":"zip"},inplace=True)
rent["zip"] = rent["zip"].astype(str).str.zfill(5)
rent.sort_values("zip",inplace=True)

rent.set_index("zip",inplace=True)

In [193]:
rent = pd.DataFrame(rent.iloc[:,-5:].mean(axis=1))

In [194]:
rent.columns = ["rent"]

In [195]:
rent.reset_index(inplace=True)

In [196]:
regr=pd.merge(rent,city,on="zip")

In [197]:
regr = pd.merge(regr,area,on="zip")

In [199]:
from sklearn import linear_model
import statsmodels.api as sm

In [204]:
regr = regr[["zip","rent","population","density","area"]]

In [205]:
x = regr.columns[2:]
y = regr.columns[1]
X = regr[x]
Y = regr[y]

lm = linear_model.LinearRegression()
lm.fit(X, Y)
r_sq = lm.score(X,Y)

intercept = lm.intercept_
coef = list(lm.coef_)
regr_df = pd.DataFrame()
index = ["R^2","intercept"] + list(x)
coef = [r_sq,intercept] + coef
regr_df["index"] = index
regr_df["coef"] = coef

In [208]:
regr_df

,index,coef
0,R^2,0.297379
1,intercept,0.963605
2,population,0.000001
3,density,0.000045
4,area,-0.000268


In [78]:
# city = pd.read_csv("zip_code_city.csv",dtype={"Zip":"str"})
# zip_df1 = pd.merge(zip_df,city,on="Zip")
# zip_df1